# P300 Speller – Offline & Pseudo-Online Demo

This notebook demonstrates a complete P300 BCI pipeline:
- EEG loading
- Preprocessing & epoching
- CNN-based P300 detection
- Character decoding (pseudo-online)

⚠️ Instructions:
1. Restart kernel
2. Run all cells
3. Observe decoded word and live demo


In [1]:
# ===== Core Imports =====
import numpy as np
import os
import time
import scipy.io as sio
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix

import tensorflow as tf


In [2]:
#Load the saved model
model = tf.keras.models.load_model(
    "../models/p300_cnn.keras",
    compile=False
)
print("✅ Pretrained P300 CNN model loaded")


✅ Pretrained P300 CNN model loaded


In [3]:
#Load preprocessed test data
X_test = np.load("../data/X_test.npy")
y_test = np.load("../data/y_test.npy")
stim_test = np.load("../data/stim_test.npy")

print(X_test.shape, y_test.shape, stim_test.shape)


(2592, 64, 192, 1) (2592,) (2592,)


In [4]:
y_prob = model.predict(X_test).ravel()


81/81 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step


In [5]:
#Decodes one character from accumulated probabilities.
import numpy as np

def decode_char_from_probs(stim_codes, probs):
    """
    Decode a single character from P300 probabilities.
    stim_codes: stimulus codes (1–12)
    probs: predicted probabilities for each flash
    """
    row_scores = np.zeros(6)
    col_scores = np.zeros(6)

    for code, p in zip(stim_codes, probs):
        if 1 <= code <= 6:        # row flashes
            row_scores[code - 1] += p
        elif 7 <= code <= 12:     # column flashes
            col_scores[code - 7] += p

    row = np.argmax(row_scores)
    col = np.argmax(col_scores)

    return row, col


In [6]:
#Decodes a full word / sequence of characters.
def decode_word(stim_codes, probs, chars=5, flashes_per_char=120):
    """
    Decode multiple characters sequentially.
    """
    decoded = []
    pointer = 0

    for _ in range(chars):
        chunk_codes = stim_codes[pointer:pointer + flashes_per_char]
        chunk_probs = probs[pointer:pointer + flashes_per_char]

        row, col = decode_char_from_probs(chunk_codes, chunk_probs)
        decoded.append(speller[row, col])

        pointer += flashes_per_char

    return "".join(decoded)


In [7]:
#Pseudo-online real-time style demo
import time

def live_p300_demo(stim_codes, probs, flashes_per_char=120, delay=0.05, max_chars=5):
    """
    Simulate an online P300 speller demo.
    """
    row_scores = np.zeros(6)
    col_scores = np.zeros(6)

    flash_count = 0
    char_count = 0

    for code, p in zip(stim_codes, probs):
        time.sleep(delay)

        if 1 <= code <= 6:
            row_scores[code - 1] += p
        elif 7 <= code <= 12:
            col_scores[code - 7] += p

        flash_count += 1

        if flash_count == flashes_per_char:
            row = np.argmax(row_scores)
            col = np.argmax(col_scores)
            char = speller[row, col]

            print("🧠 Spelled character:", char)

            row_scores[:] = 0
            col_scores[:] = 0
            flash_count = 0
            char_count += 1

            if char_count == max_chars:
                print("✅ Demo finished")
                break


In [8]:
speller = np.array([
    ['A','B','C','D','E','F'],
    ['G','H','I','J','K','L'],
    ['M','N','O','P','Q','R'],
    ['S','T','U','V','W','X'],
    ['Y','Z','1','2','3','4'],
    ['5','6','7','8','9','_']
])


In [9]:
decoded_word = decode_word(
    stim_test,
    y_prob,
    chars=5,
    flashes_per_char=120
)

print("🧠 Decoded word:", decoded_word)


🧠 Decoded word: JPAMG


In [10]:
live_p300_demo(stim_test, y_prob, max_chars=8)

🧠 Spelled character: J
🧠 Spelled character: P
🧠 Spelled character: A
🧠 Spelled character: M
🧠 Spelled character: G
🧠 Spelled character: P
🧠 Spelled character: D
🧠 Spelled character: G
✅ Demo finished
